# Création des resources RDF des mentions de parcelles

In [33]:
import pandas as pd
import numpy as np
import uuid
import re
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
from rdflib.namespace import XSD, DCTERMS, PROV, SKOS
from functions import *

## 1. Matrice de 1813

In [34]:
COMMUNE = 'Gentilly'
PLAN = '1810'
MATRICE_ID = 'MAT_B_NB_1813'

In [35]:
PATH = "/home/STual/KG-cadastre/data/gentilly/MAT_1813.csv"
OUTPUT_FOLDER_PATH = "/home/STual/KG-cadastre/data/rdf"
mat1813 = pd.read_csv(PATH,header=0)
display(mat1813)

,ID,Type_CF,Num_Folio,Ordre de lecture,Image,Section_clean,Parcelle_clean,Lieu-dit_transcript,Lieu-dit_clean,Propriétaires_transcript,...,Nature_clean,Date entrée (officieuse),Date sortie (officieuse),Tiré de,Porté à,Ligne barrée ?,CF rayé ?,Spécification,Commentaire,Cote liée
0,1,Bâti,108,NaN,FRAD094_3P_000255_01_0586,B,226,NaN,NaN,Legendre H↑re↓ de →Fontainebleau,...,maison,NaN,NaN,NaN,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN
1,2,Bâti,114,NaN,FRAD094_3P_000255_01_0588,B,227,NaN,NaN,Louves,...,maison,NaN,1814.0,NaN,82,Oui,Oui,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN
2,3,Bâti,82,NaN,FRAD094_3P_000255_01_0579,B,227,NaN,NaN,"Hardon, Bourgeois ~~nourisseur~~→à Paris",...,maison,1814.0,NaN,114,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN
3,4,Bâti,82,NaN,FRAD094_3P_000255_01_0579,B,228,NaN,NaN,"Hardon, Bourgeois ~~nourisseur~~→à Paris",...,maison,NaN,NaN,NaN,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN
4,5,Bâti,64,NaN,FRAD094_3P_000255_01_0571,B,231,NaN,NaN,Faipot François→m↑d↓ de vin b↑re↓ de fontaineb...,...,maison,NaN,NaN,NaN,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,67,Non bâti,440,1.0,FRAD094_3P_000255_01_0509,B,224,id,Les girantiers,Thiersse→à paris,...,terre,NaN,1836.0,NaN,"236↑4↓,361,258→166,235²,138,357,440",Oui,Non,NaN,"Dans Porté à, sur le registre on a tendance à ...",NaN
67,68,Non bâti,440,1.0,FRAD094_3P_000255_01_0509,B,224bis,Les Girantiers,Les girantiers,Thiersse→à paris,...,terre,1836.0,NaN,Reste,NaN,Non,Non,NaN,NaN,NaN
68,69,Non bâti,446,2.0,FRAD094_3P_000255_01_0516,B,263,rue thiers,rue thiers,Thomsain→Jacques Joseph,...,maison,1831.0,NaN,C↑ons↓ N↑elle↓,NaN,NaN,NaN,NaN,NaN,NaN
69,70,Non bâti,455,1.0,FRAD094_3P_000255_01_0526,B,263,rue Thiers,rue thiers,"~~Vincent, menuisier→(1829)→Desnet alexandre→r...",...,maison,NaN,NaN,C↑ons↓ N↑elle↓,NaN,Non,Non,NaN,Même maison #2,NaN


### 1.1. Création des pages
- rdf:type rico:Instanciation : instance numérisée d'une page de registre
- rdf:type rico:Record => concept de la page de registre, fait le lien avec le registre (concept, RecordSet)

In [36]:
#select distinct values in th colum Image
images = mat1813['Image'].unique()

In [37]:
# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/source/")
mlclasse = Namespace("http://data.ign.fr/id/codes/cadastre/mlClasse/")
g.bind('source', baseuri)
g.bind('mlclasse', mlclasse)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
add = Namespace("http://rdf.geohistoricaldata.org/def/address#")
rico = Namespace("https://www.ica.org/standards/RiC/ontology#")
fpo = Namespace("https://github.com/johnBradley501/FPO/raw/master/fpo.owl#")
time = Namespace("http://www.w3.org/2006/time#")

g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

for img in images:
    json = parse_record_id(img)
    subject_uri = URIRef(baseuri + f"{img}")
    g.add((subject_uri, RDF.type, rico.Instanciation))
    g.add((subject_uri, rico.identifier, Literal(img)))
    mlClasseNode = BNode()
    g.add((subject_uri, cad.hasClasse, mlClasseNode))
    g.add((mlClasseNode, cad.hasClasseValue, URIRef(mlclasse + f"MATMainTable")))
    g.add((mlClasseNode, PROV.wasGeneratedBy, URIRef(f"http://data.ign.fr/id/codes/cadastre/activity/0001")))
    folder_end = img.rfind('_')
    g.add((subject_uri,rico.isOrWasDigitalInstanciationOf,URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{img}_page")))

    subject_uri_record = URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{img}_page")
    g.add((subject_uri_record, RDF.type, rico.Record))
    g.add((subject_uri_record, rico.isOrWasIncludedIn, URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}")))

print(g.serialize(format='turtle'))
#write g into a .ttl file
g.serialize(destination=f"{OUTPUT_FOLDER_PATH}/{COMMUNE}_{MATRICE_ID}.ttl", format='turtle')

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix mlclasse: <http://data.ign.fr/id/codes/cadastre/mlClasse/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .

source:FRAD094_3P_000255_01_0015 a rico:Instanciation ;
    cad:hasClasse [ cad:hasClasseValue mlclasse:MATMainTable ;
            prov:wasGeneratedBy <http://data.ign.fr/id/codes/cadastre/activity/0001> ] ;
    rico:identifier "FRAD094_3P_000255_01_0015" ;
    rico:isOrWasDigitalInstanciationOf source:94_Gentilly_FRAD094_3P_000255_01_0015_page .

source:FRAD094_3P_000255_01_0032 a rico:Instanciation ;
    cad:hasClasse [ cad:hasClasseValue mlclasse:MATMainTable ;
            prov:wasGeneratedBy <http://data.ign.fr/id/codes/cadastre/activity/0001> ] ;
    rico:identifier "FRAD094_3P_000255_01_0032" ;
    rico:isOrWasDigitalInstanciationOf source:94_Gentilly_FRAD094_3P_000255_01_0032_page .

source:FRAD094_3P_00025

<Graph identifier=N4ce60f57da724683b9d6218a351c46bc (<class 'rdflib.graph.Graph'>)>

### 1.2 Folios
- Pré-traitement des colonnes *Num_Folio*, *Tiré de* et *Porté à*
- Création des objets "Folios" à partir de la colonne *Num_Folio* et des colonnes *Tiré de* et *Porté à* (manquants)
- Création des objets spéciaux mentionnés dans les colonnes destinées aux folios (reste, construction nouvelle, ruine etc)

In [38]:
from functions import parse_record_id, cleanNumFolio

clean_folio, clean_tire_de, clean_porte_a = [], [], []

#if elem is a list, flatten it at commas or points
symbols = [",", "→", "."," ",";"]

for index, row in mat1813.iterrows():
    clean_folio.append(cleanNumFolio(row["Num_Folio"],symbols))
    clean_tire_de.append(cleanNumFolio(row["Tiré de"],symbols))
    clean_porte_a.append(cleanNumFolio(row["Porté à"],symbols))

# Create a new column in df named "Num_Folio_clean" applying the following transformations
mat1813['Num_Folio_clean'] = clean_folio
mat1813['Tire_de_clean'] = clean_tire_de
mat1813['Porte_a_clean'] = clean_porte_a

In [39]:
#Filter sources df to keep only rows where column "Type CF" is equal to "Non bâti"
folios = mat1813[mat1813["Type_CF"] == "Non bâti"]
folios = folios.reset_index(drop=True)

In [40]:
#Create a list with unique values of "Num_Folio"
num_folios = folios["Num_Folio_clean"].unique().tolist()
num_folios = [i.split(';', 1)[0] for i in num_folios]
#Create a list with unique values of "Tiré de"
tire_de = folios["Tire_de_clean"].unique().tolist()
tire_de = [i.split(';', 1)[0] for i in tire_de]
#Create a list with unique values of "Porté à"
porte_a = folios["Porte_a_clean"].unique().tolist()
porte_a = [i.split(';', 1)[0] for i in porte_a]

Numéros de folios mentionnés dans "Tiré de" ou "Porté à" qui ne sont pas dans la colonne "Num_Folios".

In [41]:
others_num_folios = tire_de + porte_a
folios_to_search = list(set(others_num_folios)-set(num_folios))
print(sorted(folios_to_search))

['107↑2↓', '228', '238', '247ter', '274↑2↓', '46ter', 'EMPTY', 'const↑on↓ nouv↑elle↓', 'c↑n↓ n↑elle↓', 'c↑ons↓ n↑elle↓', 'c↑on↓ n↑elle↓', 'double emploi', 'reste', 'voie publique']


In [42]:
additionnal_folios = []
special_values = []

for value in folios_to_search:
    if any(char.isdigit() for char in str(value)):
        additionnal_folios.append(value)
    else:
        special_values.append(value)
print(sorted(additionnal_folios))
print(sorted(special_values))

['107↑2↓', '228', '238', '247ter', '274↑2↓', '46ter']
['EMPTY', 'const↑on↓ nouv↑elle↓', 'c↑n↓ n↑elle↓', 'c↑ons↓ n↑elle↓', 'c↑on↓ n↑elle↓', 'double emploi', 'reste', 'voie publique']


Remarques : 
- 107bis et 107↑2↓ peuvent être identiques
- 227 et 228ter peuvent être identiques
- 238 : pas de parcelles de la section B, erreur de transcription ?
- 247 et 247ter peuvent être identiques
- 274↑2↓ : pas de parcelles de la section B, erreur de transcription ?
- 46ter

In [43]:
#Create a new dataframe from sources with columns "Num_Folio" and "Image" containing only distinct rows
folios_pages = folios[["Num_Folio_clean","Image"]].drop_duplicates()
#In "Num_Folio", remove all the characters that are not digits or letters
#sources_unique["Num_Folio"] = sources_unique["Num_Folio"].str.replace(r'↓|↑', '')
display(folios_pages)

,Num_Folio_clean,Image
0,11,FRAD094_3P_000255_01_0015
3,23bis,FRAD094_3P_000255_01_0032
4,34ter,FRAD094_3P_000255_01_0044
6,46bis,FRAD094_3P_000255_01_0057
7,107bis,FRAD094_3P_000255_01_0125
9,122bis,FRAD094_3P_000255_01_0146
10,124bis,FRAD094_3P_000255_01_0148
11,138,FRAD094_3P_000255_01_0165
13,156,FRAD094_3P_000255_01_0186
15,163,FRAD094_3P_000255_01_0196


In [44]:
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
from rdflib.namespace import XSD, DCTERMS
import uuid

# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/folio/")
srcuri = Namespace("http://data.ign.fr/id/source/")
g.bind('folio', baseuri)
g.bind('source', srcuri)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
add = Namespace("http://rdf.geohistoricaldata.org/def/address#")
rico = Namespace("https://www.ica.org/standards/RiC/ontology#")
fpo = Namespace("https://github.com/johnBradley501/FPO/raw/master/fpo.owl#")
time = Namespace("http://www.w3.org/2006/time#")

g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

# Iterate over each row in the DataFrame
for index, row in folios_pages.iterrows():
    json = parse_record_id(row['Image'])
    subject_uri = URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(row['Num_Folio_clean'])}")
    g.add((subject_uri, RDF.type, cad.Folio))
    g.add((subject_uri, rico.identifier, Literal(row["Num_Folio_clean"])))
    g.add((subject_uri, DCTERMS.isPartOf, URIRef(srcuri + f'{MATRICE_ID}')))
    g.add((subject_uri, rico.isOrWasIncludedIn,URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_page")))

for folio in additionnal_folios:
    folio_uri = URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(folio)}")
    g.add((folio_uri, RDF.type, cad.Folio))
    g.add((folio_uri, rico.identifier, Literal(folio)))
    g.add((folio_uri, DCTERMS.isPartOf, URIRef(srcuri + f'{MATRICE_ID}')))

print(g.serialize(format='turtle'))

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix folio: <http://data.ign.fr/id/folio/> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .

folio:94_Gentilly_MAT_B_NB_1813_107bis a cad:Folio ;
    dcterms:isPartOf source:MAT_B_NB_1813 ;
    rico:identifier "107bis" ;
    rico:isOrWasIncludedIn source:94_Gentilly_FRAD094_3P_000255_01_0125_page .

<http://data.ign.fr/id/folio/94_Gentilly_MAT_B_NB_1813_107↑2↓> a cad:Folio ;
    dcterms:isPartOf source:MAT_B_NB_1813 ;
    rico:identifier "107↑2↓" .

folio:94_Gentilly_MAT_B_NB_1813_11 a cad:Folio ;
    dcterms:isPartOf source:MAT_B_NB_1813 ;
    rico:identifier "11" ;
    rico:isOrWasIncludedIn source:94_Gentilly_FRAD094_3P_000255_01_0015_page .

folio:94_Gentilly_MAT_B_NB_1813_122bis a cad:Folio ;
    dcterms:isPartOf source:MAT_B_NB_1813 ;
    rico:identifier "122bis" ;
    rico:isOrWasIncludedIn source:94_Gentilly_FRAD094

### 1.3 Dates
Une date dans le cadastre correspond généralement une année.

Une cellule contient une date valide si :
* uniquement des chiffres
* 4 chiffres
* valeur située entre l'initialisation et la clôture de la matrice

### 1.4 Adresses
* Création des objets adresses à partir de la colonne *Lieu-dit* (dédoublonnage simple et création des objets)
* URIS : uuid

In [45]:
# Select the column as a new DataFrame
addresses = mat1813[['Lieu-dit_clean']].copy()

# Create pre-treated columns
mat1813['Lieu-dit_treated'] = mat1813['Lieu-dit_clean']
mat1813['Lieu-dit_treated'] = mat1813['Lieu-dit_treated'].str.lower()
mat1813['Lieu-dit_treated'] = mat1813['Lieu-dit_treated'].str.replace(r'[^\w\s~]', '', regex=True)
mat1813['Lieu-dit_uuid'] = mat1813['Lieu-dit_treated'].map(dict(zip(mat1813['Lieu-dit_treated'].unique(), [uuid.uuid4() for _ in range(len(mat1813['Lieu-dit_treated'].unique()))])))

In [54]:
addresses = mat1813['Lieu-dit_treated'].drop_duplicates().values.tolist()
addresses

[nan,
 'les girantiers',
 'rue thiers',
 'les girantins',
 'rue thiers n10',
 'route de paris',
 'bre fontainebleau',
 'lauteis',
 'girantins']

In [46]:
# Create a list with all unique values of this new column
unique_addresses = mat1813[['Lieu-dit_uuid', 'Lieu-dit_treated']].drop_duplicates().values.tolist()
unique_addresses

[[UUID('caa8b110-64f0-4513-8b5f-c7a06c0d4c07'), nan],
 [UUID('87a8f430-4ffb-4d28-821f-eb61f12f6cf1'), 'les girantiers'],
 [UUID('5841e850-934e-4c57-9f49-304102dae6b4'), 'rue thiers'],
 [UUID('fabc4c66-ced6-4d28-a43a-974880671dd8'), 'les girantins'],
 [UUID('068b593f-587b-423c-81d8-a812a4838f14'), 'rue thiers n10'],
 [UUID('ae943bc9-b8db-46df-9534-4fe2c37345a0'), 'route de paris'],
 [UUID('e8bd099f-d2a2-4b37-9b07-6e2a15c4f282'), 'bre fontainebleau'],
 [UUID('3deaa773-3b2e-4d5f-a5d1-c3b1123b6af9'), 'lauteis'],
 [UUID('95ad66fc-d7cb-4c69-8ffb-0007a661ff0d'), 'girantins']]

In [31]:
# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/landmark/")
srcuri = Namespace("http://data.ign.fr/id/source/")
owneruri = Namespace("http://data.ign.fr/id/owner/")
foliouri = Namespace("http://data.ign.fr/id/folio/")
cad_ltype = Namespace("http://data.ign.fr/def/cadastre/landmarkType/")
g.bind('landmark', baseuri)
g.bind('owner', owneruri)
g.bind('source', srcuri)
g.bind('cad_ltype', cad_ltype)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
add = Namespace("http://rdf.geohistoricaldata.org/def/address#")
rico = Namespace("https://www.ica.org/standards/RiC/ontology#")
fpo = Namespace("https://github.com/johnBradley501/FPO/raw/master/fpo.owl#")
time = Namespace("http://www.w3.org/2006/time#")
g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

### 1.5 Propriétaires

### 1.6 Nature

### 1.X Création des états de parcelles

In [16]:
# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/landmark/")
srcuri = Namespace("http://data.ign.fr/id/source/")
owneruri = Namespace("http://data.ign.fr/id/owner/")
foliouri = Namespace("http://data.ign.fr/id/folio/")
cad_ltype = Namespace("http://data.ign.fr/def/cadastre/landmarkType/")
g.bind('landmark', baseuri)
g.bind('owner', owneruri)
g.bind('source', srcuri)
g.bind('cad_ltype', cad_ltype)
g.bind('folio', foliouri)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
add = Namespace("http://rdf.geohistoricaldata.org/def/address#")
rico = Namespace("https://www.ica.org/standards/RiC/ontology#")
fpo = Namespace("https://github.com/johnBradley501/FPO/raw/master/fpo.owl#")
time = Namespace("http://www.w3.org/2006/time#")

g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

# Iterate over each row in the DataFrame
for index, row in mat1813.iterrows():

    json = parse_record_id(row["Image"])
    lineuuid_ = MATRICE_ID + '_' + str(row['ID']) #str(uuid.uuid4())

    subject_uri = URIRef(baseuri + f"{json['departement']}_{COMMUNE}_{PLAN}_{row['Section_clean']}_{row['Parcelle_clean']}_{lineuuid_}")
    g.add((subject_uri, RDF.type, add.Landmark))
    g.add((subject_uri, add.isLandmarkType, cad_ltype.Plot))
    g.add((subject_uri, DCTERMS.identifier, Literal(row['Section_clean'] + '-' + row['Parcelle_clean'], datatype=XSD.string)))
    #Folios
    g.add((subject_uri, cad.hasNumFolio, URIRef(foliouri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(row['Num_Folio_clean'])}")))
    tire_de = row['Tire_de_clean'] = row['Tire_de_clean'].split(';')
    for f in tire_de:
        if any(char.isdigit() for char in str(f)):
            g.add((subject_uri, cad.takenFrom, URIRef(foliouri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(f)}")))
        elif f != 'nan':
            g.add((subject_uri, cad.takenFrom, Literal(str(f))))
    porte_a = row['Porte_a_clean'] = row['Porte_a_clean'].split(';')
    for f in porte_a:
        if any(char.isdigit() for char in str(f)):
            g.add((subject_uri, cad.passedTo, URIRef(foliouri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(f)}")))
        elif f != 'nan':
            g.add((subject_uri, cad.passedTo, Literal(str(f))))
    #Source
    rowSource = BNode()
    g.add((subject_uri, fpo.sourcedFrom, rowSource))
    g.add((rowSource, RDF.type, fpo.SourceCitation))
    g.add((rowSource, fpo.fromSource, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}")))
    g.add((rowSource, rico.isComponentOfTransitive, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}")))
    g.add((rowSource, cad.hasExtractionID, Literal(lineuuid_)))
    g.add((rowSource, PROV.wasGeneratedBy, URIRef(f"http://data.ign.fr/id/codes/cadastre/activity/0002")))
    g.add((URIRef(f"http://data.ign.fr/id/codes/cadastre/activity/0002"), PROV.used, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}")))
    g.add((rowSource, rico.isOrWasDigitalInstanciation,URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_{lineuuid_}_area")))
    g.add((rowSource, cad.lineOrderInArea, Literal(row['Ordre de lecture'], datatype=XSD.integer)))

    #Create recordpart
    recordparturi = URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_{lineuuid_}_area")
    g.add((recordparturi, RDF.type, rico.RecordPart))
    g.add((recordparturi, rico.isOrWasIncludedIn, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_page")))
    
    #Address

    #Owner

    #Nature
    
    #Time


print(g.serialize(format='turtle'))

@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix cad_ltype: <http://data.ign.fr/def/cadastre/landmarkType/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix folio: <http://data.ign.fr/id/folio/> .
@prefix fpo: <https://github.com/johnBradley501/FPO/raw/master/fpo.owl#> .
@prefix landmark: <http://data.ign.fr/id/landmark/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

landmark:94_Gentilly_1810_B_224Ptie_MAT_B_NB_1813_42 a add:Landmark ;
    cad:hasNumFolio folio:94_Gentilly_MAT_B_NB_1813_251bis ;
    cad:passedTo "EMPTY" ;
    cad:takenFrom "c↑on↓ n↑elle↓" ;
    dcterms:identifier "B-224Ptie"^^xsd:string ;
    add:isLandmarkType cad_ltype:Plot ;
    fpo:sourcedFrom [ a fpo:SourceCitation ;
            cad:hasExtractionID "MAT_B_NB_1813_42" ;
    